## LangChain에서 Splitter 유형
1. RecursiveCharacterTextSplitter
- 지능적 분할: 여러 구분자를 순서대로 시도 (\n\n → \n →   → "") : 여러 개의 separator를 가지고 recursive하게 텍스트를 분리
  * 첫 번째 separator로 분리 -> 너무 긴 청크는 다시 두 번째 separator로 분리..
- 의미 보존: 가능한 한 단어나 문장 단위로 자름
- 유연함: 다양한 텍스트 구조에 적응

2. CharacterTextSplitter
- 단순 분할: 기본적으로 \n\n(두 줄바꿈)으로만 분할
- 고정적: 지정된 구분자로만 자름
- 예측 가능: 일관된 분할 규칙

3. TokenTextSplitter
- 토큰 수를 기준으로 분할하는 방법

In [1]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)

chunk_size = 20
chunk_overlap = 5

text1 = "청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?"

/opt/anaconda3/envs/llm_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 1. RecursiveCharacterTextSplitter

In [2]:
# RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(  # ... code here
    chunk_size=chunk_size, chunk_overlap=chunk_overlap  # ... code here  # ... code here
)

r_result = r_splitter.split_text(text=text1)  # ... code here

print("RecursiveCharacterTextSplitter 결과:")
for i, chunk in enumerate(r_result):
    print(f"청크 {i+1}: '{chunk}' (길이: {len(chunk)})")

print()

RecursiveCharacterTextSplitter 결과:
청크 1: '청킹을 위한 예제 데이터입니다.' (길이: 17)
청크 2: '어떻게 쪼개지는지 알아볼까요?' (길이: 16)



## 2. CharacterTextSplitter

In [4]:
# CharacterTextSplitter

c_splitter = CharacterTextSplitter(  # ... code here
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

c_result = c_splitter.split_text(text1)

print("CharacterTextSplitter 결과:")
for i, chunk in enumerate(c_result):
    print(f"청크 {i+1}: '{chunk}' (길이: {len(chunk)})")

CharacterTextSplitter 결과:
청크 1: '청킹을 위한 예제 데이터입니다. 어떻게 쪼개지는지 알아볼까요?' (길이: 34)


In [5]:
# 한줄 띄우기, 두줄 띄우기 예시
print("오늘도")
print("\n어김없이")
print("\n\nLLM을")
print("\학습한다")

오늘도

어김없이


LLM을
\학습한다


In [6]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [7]:
c_splitter = CharacterTextSplitter(chunk_size=450, chunk_overlap=0, separator=" ")
c_splitter.split_text(some_text)

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,',
 'have a space.and words are separated by space.']

In [8]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450, chunk_overlap=0, separators=["\n\n", "\n", " ", ""]
)
r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [ ]:
#RecursiveCharacterTextSplitter의 경우, 
#가장 먼저 \n\n으로 표시된 2줄 띄어쓰기 부분을 먼저 나누고, 이때 문장 역시 공백을 기준으로 나뉜 것을 확인할 수 있다
#이후 한줄에 표시된 문장을 청크로 분리한다. 
#마지막으로 Character로 분리하기 때문에 더 세밀하게 chunking이 된다

## 3. TokenTextSplitter

In [9]:
# TokenTextSplitter 예시
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)  # ... code here

text1 = "Border collies are very intelligent, capable of learning well, and have a lot of energy, so they need a lot of exercise. They have strong herding instincts and can easily learn a variety of tricks and commands."
text_splitter.split_text(text1)

ImportError: Could not import tiktoken python package. This is needed in order to for TokenTextSplitter. Please install it with `pip install tiktoken`.

### 예제 적용

In [12]:
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기 : '스마트농업 육성사업 추친현황과개선과제.pdf', '스마트팜 기술 동향과 발전 방향.pdf'
loaders = [
    PyPDFLoader("./data/스마트농업 육성사업 추친현황과개선과제.pdf"),  # ... code here
    PyPDFLoader("./data/스마트팜 기술 동향과 발전 방향.pdf"),  # ... code here
]

docs = []
# loader에 있는 문서들을 가져와 docs 에 추가하기
for loader in loaders:
    docs.extend(loader.load())


# 전체 문서에서 page_content만 추출
for doc in docs[:3]:  # 여기서 [:2]는 첫 두 페이지만 예시로 출력
    print(doc.page_content)
    print("=" * 80)  # 구분선

스마트농업 육성사업 추진현황과 개선과제
2
0
2
2
·
6
·
1
52022. 6. 15.
스마트농업 육성사업  
추진현황과 개선과제국회예산정책처┃사업평가
Analysis on the status and  
future development of  
Smart Farming projects
변재연

스마트농업 육성사업
추진현황과 개선과제
스마트농업 육성사업 추진현황과 개선과제
총 괄Ｉ 송병철 예산분석실장
기획․조정Ｉ 서세욱 사업평가심의관
전용수 경제산업사업평가과장
작 성Ｉ 변재연 경제산업사업평가과 예산분석관
지 원Ｉ 김창민 경제산업사업평가과 자료분석지원요원
이채원 경제산업사업평가과 행정실무원
본 보고서는  ｢국회법 ｣ 제22조의2 및 ｢국회예산정책처법 ｣ 제3조에 따라 국회의원의
의정활동을 지원하기 위하여 발간되었습니다.
문의:  예산분석실 경제산업사업평가과  | 02) 6788-3777 | eie@nabo.go.kr
이 책은 국회예산정책처 홈페이지(www.nabo.go.kr)를 통하여 보실 수 있습니다.
“본 보고서는 담당 분석관의 연구 결과를 바탕으로 작성된 것으로 
국회예산정책처의 공식의견과는 다를 수 있음을 알려드립니다.”


In [24]:
from langchain.text_splitter import CharacterTextSplitter

chunk_size = 10
chunk_overlap = 5

splitter = CharacterTextSplitter.from_tiktoken_encoder(  # ... code here  # from_tiktoken_encoder을 이용하여 분리
    chunk_size=chunk_size,  # ... code here
    chunk_overlap=chunk_overlap,  # ... code here
)

doc = splitter.split_documents(docs)  # ... code here
print(f"{len(doc)}개의 문서를 로드했습니다.")

153개의 문서를 로드했습니다.


In [35]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

text = "This \n\nis a simple test sentence. \n\nIt should be split into multiple \n\nchunks."

# `from_tiktoken_encoder`를 사용하여 chunk_size 1로 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=20,
    chunk_overlap=0
)

# Document 객체 생성
docs = [Document(page_content=text)]

# 문서 분할
chunks = splitter.split_documents(docs)

# 결과 출력
print(f"원본 문서의 길이: {len(text)} 문자")
print(f"분할된 청크의 개수: {len(chunks)}개")
print("첫 5개 청크:")
for i, chunk in enumerate(chunks[:5]):
    print(f"  {i+1}: '{chunk.page_content}'")

원본 문서의 길이: 78 문자
분할된 청크의 개수: 2개
첫 5개 청크:
  1: 'This 

is a simple test sentence. 

It should be split into multiple'
  2: 'chunks.'
